# Newton-Raphson: Julia to IR

In this tutorial, we will focus on the Newton-Raphson algorithm written in Julia and create IR for the algorithm.

The sample code for the Newton-Raphson algorithm is given below:

In [ ]:
function f(x)
    (4/3)x^3 - (3/2)x^2 +8
end
function g(x)
    4x^2 - 3x
end
function newtoniteration(x)
    h = f(x) / g(x)
    while abs(h) >= 0.0001
        h = f(x) / g(x)
        x = x - h
        println("Iterations : ", x)
    end
    println("The value of the root is : ", x)
end
x0 = 5
newtoniteration(x0)

For this example, we will jump straight to the IR. You can imagine what the lexical analysis will look like by now, and we will see better ways of conceptualizing and visualizing this programs structure than a long AST.

We generate the IR with the `code_llvm` command: remember, we have to give an initial value, as Julia will emit the LLVM as it is actually running.

In [2]:
@code_llvm newtoniteration(5)

;  @ In[1]:7 within `newtoniteration`
; Function Attrs: uwtable
define void @julia_newtoniteration_985(i64 signext %0) #0 {
top:
;  @ In[1]:8 within `newtoniteration`
; ┌ @ In[1]:2 within `f`
; │┌ @ intfuncs.jl:313 within `literal_pow`
; ││┌ @ operators.jl:655 within `*` @ int.jl:88
     %1 = mul i64 %0, %0
     %2 = mul i64 %1, %0
; │└└
; │┌ @ promotion.jl:380 within `*`
; ││┌ @ promotion.jl:350 within `promote`
; │││┌ @ promotion.jl:327 within `_promote`
; ││││┌ @ number.jl:7 within `convert`
; │││││┌ @ float.jl:146 within `Float64`
        %3 = sitofp i64 %2 to double
; ││└└└└
; ││ @ promotion.jl:380 within `*` @ float.jl:405
    %4 = fmul double %3, 0x3FF5555555555555
; ││ @ promotion.jl:380 within `*`
; ││┌ @ promotion.jl:350 within `promote`
; │││┌ @ promotion.jl:327 within `_promote`
; ││││┌ @ number.jl:7 within `convert`
; │││││┌ @ float.jl:146 within `Float64`
        %5 = sitofp i64 %1 to double
; ││└└└└
; ││ @ promotion.jl:380 within `*` @ float.jl:405
    %6 = fmul double %

While Julia gives the most optimized, annotated, and user-friendly IR out of all the high-level languages we've used, at a certain point it just becomes too much. So, we turned to a different way of visualizing.

## Visualization of the IR

To visualize the IR we will use the "ShowCode" package because its easy to represent. To install the package from github you need to use this command given below:

In [ ]:
using Pkg
pkg"add https://github.com/tkf/ShowCode.jl"

We will generate LLVM with `@sc_llvm`. Once we have an LLVM code object `c`, we have various options for visualization:
``` julia
c = @sc_llvm f(args...)

c                  # view IR in the REPL
display(c)         # (ditto)
edit(c)            # open the IR in editor
print(c)           # print the IR
abspath(c)         # file path to the text containing the IR

c.native           # create native code explore
c.att              # (ditto)
c.intel            # create native code explore in intel syntax
eidt(c.native)
abspath(c.native)

c.cfg              # control-flow graph (CFG) visualizer
display(c.cfg)     # display CFG
edit(c.cfg.png)    # open PNG file in your editor
edit(c.cfg.svg)    # same for SVG
abspath(c.cfg.png) # file path to the PNG image
c.cfg_only
c.dom
```

We are most interested in the cfg.

In [ ]:
using ShowCode

c = @sc_llvm newtoniteration(5)

display(c.cfg)

With this, we can get a nice visualization of the IR.

![Newton Raphson CFG - Julia](newton_raphson_julia_cfg.png)

We can get an even further simplified version by running

In [ ]:
c.cfg_only

![Newton Raphson Simplified CFG - Julia](newton_raphson_julia_cfg_simplified.png)